In [75]:
%pip install reportlab

Note: you may need to restart the kernel to use updated packages.


In [76]:
%pip install cairosvg

Note: you may need to restart the kernel to use updated packages.


In [77]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
from reportlab.pdfbase.ttfonts import TTFont
import os
import xml.etree.ElementTree as ET
import cairosvg
import tempfile 
import shutil

In [95]:
font_path = r"C:\Windows\Fonts\Arial.ttf"
pdfmetrics.registerFont(TTFont("Arial", font_path))

# registra a fonte chinesa embutida
pdfmetrics.registerFont(UnicodeCIDFont("STSong-Light"))

# --------- 5 exemplos de caracteres (hanzi, pinyin, tradução) ----------
CHARS = [
    ("爱", "ài", "amar"),
    ("八", "bā", "oito"),
    ("爸", "bà", "pai"),
    ("吧", "ba", "partícula final"),
    ("白", "bái", "branco"),
    ("百", "bǎi", "cem"),
    ("本", "běn", "classificador de livros"),
    ("不", "bù", "não"),
    ("菜", "cài", "prato, verdura"),
    ("茶", "chá", "chá"),
    ("吃", "chī", "comer"),
    ("出", "chū", "sair"),
    ("车", "chē", "carro, veículo"),
    ("东", "dōng", "leste"),
    ("都", "dōu", "todos"),
    ("对", "duì", "correto, para"),
    ("儿", "ér", "filho/sufixo"),
    ("二", "èr", "dois"),
    ("饭", "fàn", "arroz, refeição"),
    ("馆", "guǎn", "restaurante"),
    ("飞", "fēi", "voar"),
    ("机", "jī", "máquina"),
    ("分", "fēn", "minuto, dividir"),
    ("钟", "zhōng", "relógio, minuto"),
    ("高", "gāo", "alto"),
    ("兴", "xìng", "interesse, alegria"),
    ("个", "gè", "classificador"),
    ("工", "gōng", "trabalho"),
    ("作", "zuò", "fazer"),
    ("狗", "gǒu", "cachorro"),
    ("汉", "hàn", "povo Han"),
    ("语", "yǔ", "língua"),
    ("好", "hǎo", "bom"),
    ("喝", "hē", "beber"),
    ("和", "hé", "e, com"),
    ("很", "hěn", "muito"),
    ("后", "hòu", "atrás, depois"),
    ("面", "miàn", "face, lado"),
    ("几", "jǐ", "quanto(s)"),
    ("叫", "jiào", "chamar-se"),
    ("今", "jīn", "hoje, agora"),
    ("天", "tiān", "céu, dia"),
    ("九", "jiǔ", "nove"),
    ("见", "jiàn", "ver, encontrar"),
    ("妈", "mā", "mãe"),
    ("吗", "ma", "partícula interrogativa"),
    ("买", "mǎi", "comprar"),
    ("猫", "māo", "gato"),
    ("没", "méi", "não (passado)"),
    ("关", "guān", "fechar, relação"),
    ("系", "xì", "ligar, sistema"),
    ("名", "míng", "nome"),
    ("子", "zǐ", "filho, criança"),
    ("女", "nǚ", "mulher, filha"),
    ("生", "shēng", "nascer, vida"),
    ("西", "xī", "oeste"),
    ("小", "xiǎo", "pequeno"),
    ("谢", "xiè", "agradecer"),
    ("一", "yī", "um"),
    ("中", "zhōng", "meio, centro"),
]

# --------- Parâmetros de layout ---------
PAGE_W, PAGE_H = A4

In [79]:
# Desenha o quadrado de treino em si
# parametros: 
    # c = folha 
    # x = coordenada horizontal do canto inferior esquerdo do retângulo
    # y = coordenada vertical do canto inferior esquerdo do retângulo
    # size = largura/ altura
    
def draw_mizige(c, x, y, size, dashed=True, diagonals=True):

    # desenhando um quadrado 
    c.rect(x, y, size, size)

    c.setDash(1, 2)  # traço 1, espaço 2
    # cruz
    c.line(x, y + size/2, x + size, y + size/2)
    c.line(x + size/2, y, x + size/2, y + size)

    # diagonais
    if diagonals:
        c.line(x, y, x + size, y + size)
        c.line(x, y + size, x + size, y)
    
    if dashed:
        c.setDash()  # volta ao traço contínuo
    

In [80]:
def draw_inner_char(c, cx, cy, hanzi, size_pt, mode="ghost"):
    
    if mode == "none":
        return
    if mode == "ghost":
        c.saveState()
        c.setFont('STSong-Light', size_pt)
        c.setFillColorRGB(0.75, 0.75, 0.75)
        w = pdfmetrics.stringWidth(hanzi, 'STSong-Light', size_pt)
        c.drawString(cx - w/2, cy - size_pt*0.35, hanzi)
        c.restoreState()
    elif mode == "dashed":
        c.saveState()
        text = c.beginText()
        text.setFont('STSong-Light', size_pt)
        text.setTextRenderMode(1)    # contorno apenas
        c.setLineWidth(0.8)
        c.setDash(1, 2)
        w = pdfmetrics.stringWidth(hanzi, 'STSong-Light', size_pt)
        text.setTextOrigin(cx - w/2, cy - size_pt*0.35)
        text.textLine(hanzi)
        c.drawText(text)
        c.setDash()
        c.restoreState()

In [81]:
def get_strokes_from_svg(svg_file):
    """Extrai a lista de traços (path d=...) de um SVG do makemeahanzi"""
    tree = ET.parse(svg_file)
    root = tree.getroot()
    ns = {'svg': 'http://www.w3.org/2000/svg'}
    paths = [elem.attrib['d'] for elem in root.findall('.//svg:path', ns)]
    return paths

In [82]:
def generate_step_pngs(svg_file, out_folder, size=80):
    """Gera PNGs cumulativos (traço a traço) em uma pasta temporária"""
    strokes = get_strokes_from_svg(svg_file)
    step_files = []
    for step in range(1, len(strokes)+1):
        svg_content = f'''
        <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 1024 1024">
          <g transform="scale(1,-1) translate(0,-900)">
            {''.join([f'<path d="{st}" stroke="black" stroke-width="60" fill="none"/>' for st in strokes[:step]])}
          </g>
        </svg>
        '''
        out_file = os.path.join(out_folder, f"step_{step}.png")
        cairosvg.svg2png(bytestring=svg_content.encode('utf-8'),
                         write_to=out_file,
                         output_width=size,
                         output_height=size)
        step_files.append(out_file)
    return step_files

In [111]:
def write_chars(c, chars, start_x=40, start_y=None, line_gap=70, svg_folder="svgs"):
    
    if start_y is None:
        start_y = PAGE_H - 45

    y = start_y
    for hanzi, pinyin, meaning in chars:

        # --- checa se cabe na página ---
        if y - 50 < 0:        # se não há espaço suficiente
            c.showPage()       # cria nova página
            y = PAGE_H - 45    # reseta posição no topo da página

        # escreve pinyin + tradução
        c.setFont("Arial", 11)
        texto = f"{pinyin} - {meaning} "
        c.drawString(start_x, y, texto)

        # escreve o hanzi logo abaixo
        c.setFont("STSong-Light", 23)
        c.drawString(start_x, y - 40, hanzi)

        # mede a largura do hanzi
        w = pdfmetrics.stringWidth(hanzi, "STSong-Light", 23)

        # coordenadas do quadrado (lado direito do hanzi)
        s = 1.3 * cm
        x_mizige = start_x + w + 10
        y_mizige = (y - 20) - 30   # alinha verticalmente

        for k in range(11):  # 3 quadrados
            draw_mizige(c, x_mizige + k*(s + 10), y_mizige, s,
                        dashed=True, diagonals=True)
            if k <= 3:
                draw_inner_char(c,
                                x_mizige + k*(s + 10) + s/2,
                                y_mizige + s/2,
                                hanzi,
                                int(s*0.7),
                                mode="ghost")
        
        # cria pasta temporária para os steps
        tmp_dir = tempfile.mkdtemp()
        codepoint_dec = ord(hanzi)
        svg_file = os.path.join("svgs", f"{codepoint_dec}.svg")

        if os.path.exists(svg_file):
            steps = generate_step_pngs(svg_file, tmp_dir, size=int(s))

            steps = steps[:10]
            # desenha os steps acima da fileira
            step_x = x_mizige + 330
            step_y = y_mizige + s + 8
            for step_file in steps:
                c.drawImage(step_file, step_x, step_y, width=s * 0.45, height=s * 0.45,
                            preserveAspectRatio=True, mask='auto')
                step_x += s - 20

        # apaga pasta temporária
        shutil.rmtree(tmp_dir, ignore_errors=True)
        
        y -= line_gap

In [112]:
path=r"C:\Users\beatriz\Documents\chinese_study\PDFs\teste.pdf"
c = canvas.Canvas(path, pagesize=A4)
write_chars(c, CHARS, svg_folder=r"C:\Users\beatriz\Documents\chinese_study\svgs")  # <- pasta onde estão os .svg do makemeahanzi

c.showPage()
c.save()
print(f"OK: {path}")
print("salvo")

OK: C:\Users\beatriz\Documents\chinese_study\PDFs\teste.pdf
salvo
